**本节介绍数据预处理**

In [1]:
# 导入包
import pandas as pd
import torch

: 

一，Pandas读取并处理数据

最后的目标四将全部数据数值化，方能转换到tensor，进行tensor的运算

In [108]:
# 手动生成数据集
# 路径为./data/.csv
import os
os.makedirs(os.path.join('data'), exist_ok=True)
data_file = os.path.join('data', 'house_tiny.csv')

with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [109]:
# 尝试用pandas读取数据集
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [110]:
# 将数据集分为输入输出
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
print(inputs)
print("\n")
print(outputs) # 仅1列数据的时候输出列索引及数据类型

# NaN即数据丢失

   NumRooms Alley
0       NaN  Pave
1       2.0   NaN
2       4.0   NaN
3       NaN   NaN


0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64


In [111]:
print(inputs.iloc[:,0].mean()) # 计算NumRooms的均值，发现丢失数据不处理
print(inputs['NumRooms'].mean()) # 等价形式，key索引，上面四iloc默认索引

3.0
3.0


In [112]:
inputs['NumRooms'] = inputs['NumRooms'].fillna(inputs.iloc[:,0].mean()) # 仅对某一列fillna
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [113]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


In [115]:
# 标量化
inputs = inputs.astype({"Alley_Pave":"int64","Alley_nan":"int64"})
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


二，处理好数据后转换为tensor张量格式

In [117]:
# DataFrame转换为tensor格式
X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))